การทำนายการตลาดทางตรง (Bank Marketing Prediction)
โจทย์ Binary Classification ที่ข้อมูลสะอาด แต่ท้าทายในการหาลูกค้ากลุ่มเป้าหมายที่แท้จริง

🎯 โจทย์: ธนาคารต้องการเพิ่มประสิทธิภาพของแคมเปญการตลาดทางโทรศัพท์ (Telesales) สำหรับผลิตภัณฑ์เงินฝากประจำ โดยมีข้อมูลลูกค้า (อายุ, อาชีพ, สถานะ), ข้อมูลแคมเปญในอดีต, และข้อมูลเศรษฐกิจ โจทย์คือให้ทำนายว่าลูกค้าคนใดจะ "ตอบรับ" (y=yes) ข้อเสนอ

**🔢ุดข้อมูลที่คล้ายกันสำหรับฝึกซ้อม:** **Bank Marketing Dataset**

ทำไมถึงเหมาะ: เป็นชุดข้อมูลคลาสสิกที่สมดุลดี ใช้ทดสอบทักษะการเลือก Feature และการประเมินผลโมเดลได้ครบถ้วน


In [1]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูล
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
import requests
import zipfile
import os

# URL ของชุดข้อมูล Bank Marketing (bank-additional.zip)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip'
zip_file_name = 'bank-additional.zip'
csv_file_name = 'bank-additional-full.csv'
extract_path = '/content/bank-additional/bank-additional'

# ดาวน์โหลดไฟล์ zip
print(f"Downloading {url}...")
response = requests.get(url)
with open(zip_file_name, 'wb') as f:
    f.write(response.content)
print("Download complete.")

# แตกไฟล์ zip
print(f"Extracting {zip_file_name}...")
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete.")

# อ่านไฟล์ CSV ที่ต้องการจากไฟล์ที่แตกออกมา
csv_file_path = os.path.join(extract_path, csv_file_name)
df = pd.read_csv(csv_file_path, sep=';')

# แสดงขนาดและตัวอย่างข้อมูลเพื่อตรวจสอบความถูกต้อง
print(f"\nขนาดของข้อมูล: {df.shape}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
display(df.head())

# ลบไฟล์และโฟลเดอร์ชั่วคราว
# os.remove(zip_file_name) # Uncomment this line if you want to remove the downloaded zip file
# os.remove(os.path.join(extract_path, 'bank-additional-names.txt')) # Uncomment this line if you want to remove other files
# os.remove(os.path.join(extract_path, 'bank-additional.csv')) # Uncomment this line if you want to remove other files
# os.remove(os.path.join(extract_path, '.Rhistory')) # Uncomment this line if you want to remove other files
# if os.path.exists(os.path.join(extract_path, '.DS_Store')): # Uncomment this block if you want to remove .DS_Store
#     os.remove(os.path.join(extract_path, '.DS_Store'))
# if os.path.exists(os.path.join('__MACOSX', extract_path, '._.DS_Store')): # Uncomment this block if you want to remove ._DS_Store
#      os.remove(os.path.join('__MACOSX', extract_path, '._.DS_Store'))
# if os.path.exists(os.path.join('__MACOSX', extract_path)): # Uncomment this block if you want to remove __MACOSX/bank-additional
#     os.rmdir(os.path.join('__MACOSX', extract_path))
# if os.path.exists('__MACOSX'): # Uncomment this block if you want to remove __MACOSX
#      os.rmdir('__MACOSX')
# os.rmdir(extract_path) # Uncomment this line if you want to remove the extracted folder

Download complete.
Extracting bank-additional.zip...
Extraction complete.

ขนาดของข้อมูล: (41188, 21)

ตัวอย่างข้อมูล 5 แถวแรก:


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
# แบ่งข้อมูลเป็น train (80%) และ test (20%) เพื่อจำลองการแข่งขัน
# random_state=42 เพื่อให้สามารถทำซ้ำผลลัพธ์เดิมได้ทุกครั้ง
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

# เก็บ index ของ test_data ไว้สำหรับสร้างไฟล์ submission จำลอง
test_indices = test_data.index

# ลบคอลัมน์คำตอบ (y) ออกจาก test_data
test_data_no_label = test_data.drop(columns=['y'])

print(f"ขนาดข้อมูลสำหรับ Train: {train_data.shape}")
print(f"ขนาดข้อมูลสำหรับ Test:  {test_data_no_label.shape}")


ขนาดข้อมูลสำหรับ Train: (32950, 21)
ขนาดข้อมูลสำหรับ Test:  (8238, 20)


In [6]:
# กำหนดคอลัมน์เป้าหมาย
label = 'y'

# กำหนดเวลาในการฝึก (หน่วยเป็นวินาที), 300 วินาที = 5 นาที
time_limit = 300

# สร้าง Predictor
predictor = TabularPredictor(
    label=label,
    eval_metric='f1',
    path='ag_models_bank_marketing'
)

# เริ่มฝึกโมเดล
predictor.fit(
    train_data,
    time_limit=time_limit,
    presets='best_quality'
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.33 GB / 12.67 GB (89.4%)
Disk Space Avail:   65.01 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

In [7]:
# แสดง Leaderboard ของโมเดลทั้งหมด
# สำหรับ F1-Score ค่ายิ่งสูงยิ่งดี
predictor.leaderboard(train_data, silent=True)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,0.999459,0.523127,f1,1.471360,1.410662,0.144634,1.471360,1.410662,0.144634,1,True,2
1,LightGBMXT_BAG_L1,0.715110,0.601916,f1,14.799995,3.109210,73.528152,14.799995,3.109210,73.528152,1,True,3
2,LightGBMXT_BAG_L2,0.710615,0.611816,f1,24.106470,8.506873,193.443643,3.488168,1.805892,59.772528,2,True,6
3,WeightedEnsemble_L3,0.710615,0.611816,f1,24.109783,8.517901,194.845179,0.003313,0.011029,1.401536,3,True,7
4,LightGBM_BAG_L1,0.710574,0.608785,f1,2.759940,0.774323,59.856794,2.759940,0.774323,59.856794,1,True,4
5,WeightedEnsemble_L2,0.710574,0.608785,f1,2.764669,0.784792,60.739219,0.004729,0.010469,0.882425,2,True,5
6,KNeighborsUnif_BAG_L1,0.701095,0.533273,f1,1.587007,1.406786,0.141535,1.587007,1.406786,0.141535,1,True,1


In [8]:
# ทำนายผลบนข้อมูล test
predictions = predictor.predict(test_data_no_label)

# สร้าง DataFrame สำหรับไฟล์ submission
submission = pd.DataFrame({
    'id': test_indices,
    'y': predictions
})

# แสดงตัวอย่าง 5 แถวแรกของไฟล์ submission
print("ตัวอย่างไฟล์ Submission:")
submission.head()

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_bank_marketing.csv', index=False)

print("\nสร้างไฟล์ submission_bank_marketing.csv เรียบร้อยแล้ว!")


ตัวอย่างไฟล์ Submission:

สร้างไฟล์ submission_bank_marketing.csv เรียบร้อยแล้ว!


In [10]:
submission['y'].value_counts()

,count
y,
no,6898
yes,1340


In [11]:
submission = submission.rename(columns={'y': 'pred'})
submission.head()

,id,pred
2,2,no
9,9,no
10,10,no
12,12,no
16,16,no


In [12]:
submission

,id,pred
2,2,no
9,9,no
10,10,no
12,12,no
16,16,no
...,...,...
41171,41171,no
41172,41172,yes
41176,41176,yes
41181,41181,yes
